In [5]:
import pandas as pd
import pprint
import matplotlib.pyplot as plt
from random import randint
import datetime
from dateutil.relativedelta import relativedelta

# Local imports
from data_fetcher import DataFetcher
from preprocessing import Processor

In [6]:
# Define some string constants for easy typing
SAMPLE_DATA_BY_SITE = 'sampleData/bySite'
SAMPLE_DATA_BY_COUNTY = 'sampleData/byCounty'
SAMPLE_DATA_BY_STATE = 'sampleData/byState'
SAMPLE_DATA_BY_BOX = 'sampleData/byBox'
SAMPLE_DATA_BY_CBSA = 'sampleData/byCBSA'

LIST_STATES = 'list/states'
LIST_COUNTIES_BY_STATE = 'list/countiesByState'
LIST_SITES_BY_COUNTY = 'list/sitesByCounty'
LIST_CBSAs = 'list/cbsas'
LIST_PARAM_CLASSES = 'list/classes'
LIST_PARAM_IN_CLASS = 'list/parametersByClass'

In [8]:
datafetcher = DataFetcher()

In [9]:
cali_code = datafetcher.get_codes(LIST_STATES, all=False, value='California')
print('California state code:', cali_code)

criteria_code = datafetcher.get_codes(LIST_PARAM_CLASSES, all=False, value='Criteria Pollutants')
print('Criteria polutants code:', criteria_code)

carbon_monoxide_code = datafetcher.get_codes(LIST_PARAM_IN_CLASS, all=False, value='Carbon monoxide', nparams={'pc':criteria_code})
print('Carbon monoxide code:', carbon_monoxide_code)

California state code: 06
Criteria polutants code: CRITERIA
Carbon monoxide code: 42101


In [10]:
cali_sample_df = datafetcher.get_data(SAMPLE_DATA_BY_STATE, carbon_monoxide_code, 20180101, 20180115, df=True, nparams={'state':cali_code})


los_angeles_code = datafetcher.get_codes(LIST_COUNTIES_BY_STATE, all=False, value='Los Angeles', nparams={'state':cali_code})
print("Los Angeles county code: ", los_angeles_code)

pomona_code = datafetcher.get_codes(LIST_SITES_BY_COUNTY, all=False, value='Pomona', nparams={'state':cali_code, 'county':los_angeles_code})
print("Pomona site code: ", pomona_code)

pomona_site_json = datafetcher.get_data(SAMPLE_DATA_BY_SITE, carbon_monoxide_code, 20180101, 20181231, nparams={'state':cali_code, 'county':los_angeles_code, 'site': pomona_code})
pomona_site_df = pd.DataFrame(pomona_site_json)
pomona_site_df.head()

NameError: name 'get_data' is not defined